In [1]:
## De functie die tf-records leest en ze uiteindelijk weer wegschrijft (maar dan met minder labels) heet readTfRecord
## Deze functie roept andere functies aan: getLabels en getLabelIndices

import tensorflow as tf
import numpy as np
import pandas as pd
import re
import os
from collections import Counter
from tensorflow import gfile


/home/hugo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# path
#train_data_pattern = '/home/hugo/jupyter-workspace/data/audioset/audioset_v1_embeddings/bal_train/*.tfrecord'
train_data_pattern = '/home/hugo/jupyter-workspace/data/audioset/audioset_v1_embeddings/bal_train/*.tfrecord'
train_data_folder_new = '/home/hugo/jupyter-workspace/data/audioset/audioset_v1_embeddings/bal_train_new_labels'
# test set
test_data_pattern = '/home/hugo/jupyter-workspace/data/audioset/audioset_v1_embeddings/eval/*.tfrecord'
test_data_folder_new = '/home/hugo/jupyter-workspace/data/audioset/audioset_v1_embeddings/eval_new_labels'
# unbalanced set
unb_data_pattern = '/home/hugo/jupyter-workspace/data/audioset/audioset_v1_embeddings/unbal_train/*.tfrecord'
unb_data_folder_new = '/home/hugo/jupyter-workspace/data/audioset/audioset_v1_embeddings/unbal_train_new_labels'

In [3]:
write_tfrecords_without_labels = False

In [4]:
## Keep list of all new labels (to see which ones occur often)
all_train_labels = []
all_test_labels = []
all_unbal_labels = []

In [5]:
## Open old csv file and alternated csv file with the super classes
mid_to_label_old = pd.read_csv('audioset/class_labels_indices.csv')


In [6]:
mid_to_label_new = pd.read_csv('audioset/class_labels_indices_amsterdam2.csv',quotechar = '"')


In [7]:
#### Find new label for each of old label (527 -> 31)

def getNewLabel(label_old):
    ## Get m-id that matches with index
    mid = mid_to_label_old.loc[mid_to_label_old['index'] == label_old, 'mid']
    #print('mid : ',mid)

    ## Find the new label belonging to this m-id
    label_new = mid_to_label_new.loc[mid_to_label_new['mid'] == str(mid.values[0]), 'index']
    #print('label_new : ',label_new)
    
    return label_new

In [8]:
#### Find new label for each of old label (527 -> 31)

def getDisplayName(index):
    ## Find the new label belonging to this index
    return mid_to_label_new.loc[index]['display_name']
    

In [9]:
def createExample(example_old, new_labels):
    
    #print("New labels: " + str(new_labels))
    
    audio_embedding = example_old.feature_lists.feature_list['audio_embedding']
    feature_lists = tf.train.FeatureLists(feature_list={"audio_embedding": audio_embedding})

    label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=new_labels))
    video_id = example_old.context.feature['video_id'].bytes_list.value
    videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id)) 

    context_feats = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat})

    sequence_example = tf.train.SequenceExample(context=context_feats, feature_lists=feature_lists)
    
    return sequence_example

In [10]:
#### Lees tf-records en iterate over de examples#### Lee 
#### Voor elk example wordt de video-id gepakt en de bijbehorende lijst m-id's gezocht (in balanced_segment.csv)

## One tf-record has several examples
## This function checks for each example if it has a label we are interested in
## If so, we compute the one-hot vector
## Input: filename of tf-record
## Result: write new tf-record

def readWriteTfRecord(tfrecords_filename, new_folder, t):
    
    ## Create writer that writes new tf-record in train_data_folder_new
    path_tfrecord_new = new_folder + '/' + tfrecords_filename.split('/')[-1]
    writer = tf.python_io.TFRecordWriter(path_tfrecord_new)
    nr_new_examples = 0
    
    #print(path_tfrecord_new)
    ## Iterator that goes through all examples of tf-record
    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)
    
    for string_record in record_iterator:
        
        ## Take one example
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
 
        ## Get labels and find for each label the m-code in class_label_indices_old
        labels = example.context.feature['labels'].int64_list.value
        #print('labels : ', labels)
        
        new_labels = []
        for label in labels:
            ## hij returnt een set van cellen (of leeg of van 1 rij)
            label_new = getNewLabel(label)
            
            if(label_new.shape[0]>0):
                new_labels.append(label_new.values[0])
                #print('old : ',label,' new : ',label_new)
        ## If any of the labels left (for our model), write new tf-record
        if len(new_labels) > 0 or write_tfrecords_without_labels:

            ## Keep track
            if t == 'train' :
                all_train_labels.extend(new_labels)
            elif t == 'test':
                all_test_labels.extend(new_labels)
            else:
                all_unbal_labels.extend(new_labels)
            
            sequence_example = createExample(example, new_labels)
            nr_new_examples += 1
            
            ## Write the example
            writer.write(sequence_example.SerializeToString())
            #print('Example gemaakt! Video-id: ' + str(example.context.feature['video_id'].bytes_list.value))

    #print("nr new examples: " + str(nr_new_examples))
    # Only write if any examples created

    if nr_new_examples > 0:
        writer.close()
        #print("tfrecord written")
    else:
        #print("empty file.. remove: " + path_tfrecord_new)
        os.remove(path_tfrecord_new)
        
    return

In [11]:
# train_data_pattern = 'tfrecords/train_data_old/*.tfrecord'
# progres log
from log_progress import log_progress

## all train files
files = gfile.Glob(train_data_pattern) 

for file in log_progress(files, every=50):
    ## Read a file and rewrite it
    readWriteTfRecord(file, train_data_folder_new, 'train')
    

In [12]:
# print labels and counts
cnt_labels_train = Counter(all_train_labels)

for i in cnt_labels_train:
    print(getDisplayName(i),' : ',cnt_labels_train[i])
    

Tools  :  101
Gunshot, gunfire  :  178
Emergency vehicle  :  141
Fire engine, fire truck (siren)  :  60
Vehicle horn, car horn, honking  :  60
Accelerating, revving, vroom  :  60
Aircraft  :  109
Train wheels squealing  :  60
Boat, Water vehicle  :  242
Police car (siren)  :  61
Ambulance (siren)  :  69
Dog  :  273
Bark  :  60
Howl  :  60
Jackhammer  :  60
Chainsaw  :  60
Bus  :  59
Car passing by  :  61
Motor vehicle (road)  :  63
Idling  :  60
Engine starting  :  60
Motorcycle  :  60


In [13]:
# progres log
from log_progress import log_progress
## Read and rewrite all test files
files = gfile.Glob(test_data_pattern)

for file in log_progress(files, every=10):    
    al = readWriteTfRecord(file, test_data_folder_new,'test')

In [14]:
cnt_labels_test = Counter(all_test_labels)
for i in cnt_labels_test:
    print(getDisplayName(i),' : ',cnt_labels_test[i])

Motorcycle  :  61
Accelerating, revving, vroom  :  151
Chainsaw  :  60
Emergency vehicle  :  144
Police car (siren)  :  60
Bus  :  60
Idling  :  92
Boat, Water vehicle  :  239
Tools  :  100
Dog  :  230
Aircraft  :  113
Gunshot, gunfire  :  174
Bark  :  60
Car passing by  :  60
Jackhammer  :  60
Engine starting  :  60
Vehicle horn, car horn, honking  :  60
Howl  :  60
Fire engine, fire truck (siren)  :  61
Motor vehicle (road)  :  90
Ambulance (siren)  :  65
Train wheels squealing  :  60


In [15]:
## Read and rewrite all unbalanced files

# progres log
from log_progress import log_progress

files = gfile.Glob(unb_data_pattern)

for file in log_progress(files, every = 5):
    
    ## Read a file and rewrite it
    al = readWriteTfRecord(file, unb_data_folder_new, 'unbal')

In [16]:
cnt_unb_labels = Counter(all_unbal_labels)
for i in cnt_unb_labels:
    print(getDisplayName(i),' : ',cnt_unb_labels[i])

Accelerating, revving, vroom  :  11827
Vehicle horn, car horn, honking  :  3587
Motor vehicle (road)  :  8891
Chainsaw  :  1667
Car passing by  :  3684
Train wheels squealing  :  382
Gunshot, gunfire  :  3869
Boat, Water vehicle  :  13090
Fire engine, fire truck (siren)  :  3078
Idling  :  6089
Tools  :  7906
Dog  :  13202
Bark  :  2512
Engine starting  :  988
Motorcycle  :  7140
Aircraft  :  5254
Emergency vehicle  :  5445
Police car (siren)  :  3538
Bus  :  5171
Howl  :  718
Ambulance (siren)  :  1805
Jackhammer  :  161


In [ ]:
###########################################################################################
# Helper: show Examples in tfrecord file (only run this if you want to see just one example)
filename = train_data_folder_new + '/-1.tfrecord'
#filename = train_data_pattern.split('/*')[0] + '/-1.tfrecord'
record_iterator = tf.python_io.tf_record_iterator(path=filename)

# [137, 220, 519]
# [135, 137]
# [27, 466]
# [137, 264]

for string_record in record_iterator:

    ## Take one example
    example = tf.train.SequenceExample()
    example.ParseFromString(string_record)
    print(example.context.feature['labels'].int64_list.value)